# Introduction au package KERAS et au "Deep-Learning": réseaux de neurones convolutifs  (CNNs) & reconnaissance d'images

Dans ce notebook nous allons apprendre à manipuler les fonctions de base du [package Keras](https://keras.io/) permettant de mettre en oeuvre des algorithmes de réseaux de neurones, dits de "deep learning". Nous considérerons pour cela le problème le plus classique : catégorisation d'images par réseaux de neurones convolutifs (CNNs : Convolutional Neural Networks).
Nous travaillerons sur la base MNIST qui présente l'avantage (1) d'être de taille relativement restreinte, ce qui permet de réaliser l'analyse dans un temps raisonnable, et (2) d'être déjà disponible dans Keras, ce qui permet de gagner du temps sur des questions de mise en forme des données.

Le plan sera le suivant  :
1. **préparation des données**
2. **réseaux de neurones à une couche & régression (logistique) multinomiale**
3. **réseaux de neurones multicouches & modèles non-linéaires**
4. **réseaux de neurones convolutifs & apprentissage de représentation**

On commence par charger les packages et par bloquer la graine du générateur aléatoire par souci de reproductibilité. 
**Attention : keras faisant partie de tensorflow depuis la version 2 de tensorflow, les imports ne seront pas les mêmes si vous travaillez à partir de la version 2 ou d'une version antérieure.**
Pour cela, renseigner la variable **"tensorflow_version"** dans la cellule ci-dessous à la valeur **"V1"** ou **"V2"** en fonction de votre configuration.

In [ ]:
# specify tensorflow version
tensorflow_version = "V2"

In [ ]:
# import packages
import csv
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle
# import tensorflow requirements - depending on version
if tensorflow_version == "V1":
    from keras import backend as K
    from keras.models import Model
    from keras.models import Sequential
    from keras.models import load_model
    from keras.layers import Dense, Activation, Flatten
    from keras.layers import Conv2D, MaxPooling2D
    from keras.utils import to_categorical
    from keras.datasets import mnist
if tensorflow_version == "V2":
    from tensorflow.keras.models import Model
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.models import load_model
    from tensorflow.keras.layers import Dense, Activation, Flatten
    from tensorflow.keras.layers import Conv2D, MaxPooling2D
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.datasets import mnist
# set random seed
np.random.seed(123)

# Partie 1. Préparation des données

Pour préparer les données, on va commencer par les charger puis construire quelques objets nécessaires pour Keras.

## Chargement des données

Pour charger les données on s'appuie sur la fonction **mnist.load_data()** proposée par Keras.

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
print('size of training data: ')
print(X_train.shape)
print('size of test data: ')
print(X_test.shape)

On dispose donc de 60000 instances d'apprentissage et de 10000 instances de test. Chaque instance est une image de taille 28x28. La figure ci-dessous en représente deux. Notons qu'elle sont représentées en niveaux de gris (i.e., avec des intensités entre 0 et 255 car codées ici sur 8 bits).

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(X_train[27], cmap = 'gray')
plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(X_test[42], cmap = 'gray')
plt.colorbar()
plt.show()

La figure suivante donne la constitution du jeu de données d'apprentissage selon les différentes catégories de chiffres.

In [ ]:
counts = np.bincount(y_train)
values = np.unique(y_train) 
plt.bar(values, counts)
plt.xticks(range(len(values)), values)
plt.title('training dataset constitution')
plt.show()

## Transformation des données

Avant de pouvoir appliquer Keras, il convient d'appliquer quelques transformations à nos données.

On commence par **normaliser les images**: on ramène les intensités des pixels entre 0 et 1 en les divisant par 255, la valeur maximale qu'elles peuvent atteindre $(255 = 2^8-1$ car codées sur 8 bits). C'est une opération est classique en traitement d'images.

In [ ]:
# convert our data type to float32 and normalize our data values to the range [0, 1]
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

On peut vérifier que cela n'a pas transformé la structure des images.

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(X_train[27], cmap = 'gray')
plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(X_test[42], cmap = 'gray')
plt.colorbar()
plt.show()

Ensuite, notre problème étant un problème de classification multiclasse, il convient d'appliquer une transformation de type "one hot encoding" à notre variable de sortie.

In [ ]:
# convert to one-hot encoding
Y_train = to_categorical(y_train)
Y_test  = to_categorical(y_test)

In [ ]:
print('original (vector) encoding:')
print(y_train[0:3])
print('one-hot (matrix) encoding:')
print(Y_train[0:3,:])

## Sauvegarde des données

In [ ]:
dataset = [X_train, X_test, Y_train, Y_test, y_train, y_test]
f = open('mnist-dataset.pickle', 'wb')
pickle.dump(dataset, f)
f.close()